## Concat

### Import

In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import re
import datetime as dt

# # visualization
# import seaborn as sns
# import matplotlib.pyplot as plt
# %matplotlib inline

# Writing multiple dataframes to worksheets using Pandas and XlsxWriter
import xlsxwriter
import glob

### Naming

In [2]:
try:
    xls= pd.ExcelFile(r'C:\Users\u1-510\Desktop\Model_Time\Naming\Google_naming.xlsx')
    naming = pd.read_excel(xls, 'Google_nameng')
    replace_naming = pd.read_excel(xls, 'Replace')
    df_app = pd.read_excel(xls, 'App')
except:
    print('Файл отсутствует')

In [3]:
replace_naming.drop_duplicates(subset=['Stock_Campaign'], keep='last', inplace = True)

In [4]:
df_app.drop_duplicates(subset=['App'], inplace = True)

In [5]:
# Замена пробелов
replace_naming['Stock_Campaign'].replace(r'\s+','',regex=True, inplace=True)
replace_naming['Replace'].replace(r'\s+','',regex=True, inplace=True)

In [6]:
Google_nameng = replace_naming['Replace'].tolist()
Stock_Campaign = replace_naming['Stock_Campaign'].tolist()

### Reading files

In [7]:
df_cost = pd.concat([pd.read_csv(f, sep='\t', skiprows=2, 
                                 encoding='UTF-16 LE', header = None) for f in 
                     glob.glob(r'C:\Users\u1-510\Desktop\Model_Time\Join\cost*.csv')])

In [8]:
df_result = pd.concat([pd.read_csv(f, sep='\t', skiprows=2, 
                                   encoding='UTF-16 LE', header = None) for f in 
                       glob.glob(r'C:\Users\u1-510\Desktop\Model_Time\Join\result*.csv')])

In [9]:
сols = list(df_cost.columns)
сols[0] = 'Campaign'
сols[1] = 'Ad_group'
сols[2] = 'Date'
сols[3] = 'Ad_group_target_CPA'
сols[4] = 'Network'
сols[5] = 'Currency'
сols[6] = 'Cost'
сols[7] = 'Show'
сols[8] = 'Clicks'
df_cost.columns = сols

In [10]:
сols_result = list(df_result.columns)
сols_result[0] = 'Action_conversion'
сols_result[1] = 'Campaign'
сols_result[2] = 'Ad_group'
сols_result[3] = 'Date'
сols_result[4] = 'Ad_group_target_CPA'
сols_result[5] = 'Network'
сols_result[6] = 'Currency'
сols_result[7] = 'All_conv'
df_result.columns = сols_result

In [11]:
# Фильтр заголовков
df_cost = df_cost[df_cost['Campaign'].map(len) > 20]
df_result = df_result[df_result['Campaign'].map(len) > 20]

### Changing formats

In [14]:
# Нижний регистр
df_result['Action_conversion'] = df_result['Action_conversion'].str.lower()

In [15]:
# Сбросить индексы
df_result.reset_index(drop=True, inplace=True)
df_cost.reset_index(drop=True, inplace=True)

In [16]:
# Замена значений
df_result['Action_conversion'].loc[df_result['Action_conversion']
                                   .str.contains(r'\bвстановлення додатк|встановлення|установк|instal|installs\b')] = 'Install'
df_result['Action_conversion'].loc[df_result['Action_conversion'].str.contains(r'\bsession_start\b')] = 'Session start'
df_result['Action_conversion'].loc[df_result['Action_conversion'].str.contains(r'\bfirst_open\b')] = 'First open'
df_result['Action_conversion'].loc[df_result['Action_conversion'].str.contains(r'\bsign_up\b')] = 'Registration'
df_result['Action_conversion'].loc[df_result['Action_conversion'].str.contains(r'\bin_app_purchase\b')] = 'FTD'

In [17]:
# Изменение форматов 
df_result['Ad_group_target_CPA'].replace(r'\s+','',regex=True, inplace=True)
df_result['Ad_group_target_CPA'] = df_result['Ad_group_target_CPA'].apply(lambda x: x.replace(',','.'))
df_result['Ad_group_target_CPA'] = df_result['Ad_group_target_CPA'].astype(float)
df_result['Ad_group_target_CPA'] = df_result['Ad_group_target_CPA'].astype(int)
df_result['Ad_group_target_CPA'] = df_result['Ad_group_target_CPA'].astype(str)

df_result['All_conv'].replace(r'\s+','',regex=True, inplace=True)
df_result['All_conv'] = df_result['All_conv'].apply(lambda x: x.replace(',','.'))
df_result['All_conv'] = df_result['All_conv'].astype(float)
df_result['All_conv'] = df_result['All_conv'].astype(int)

In [18]:
# Замена пробелов
df_cost['Show'].replace(r'\s+','',regex=True, inplace=True)
df_cost['Cost'].replace(r'\s+','',regex=True, inplace=True)
df_cost['Clicks'].replace(r'\s+','',regex=True, inplace=True)

In [19]:
# Изменение форматов
df_cost['Ad_group_target_CPA'].replace(r'\s+','',regex=True, inplace=True)
df_cost['Ad_group_target_CPA'] = df_cost['Ad_group_target_CPA'].apply(lambda x: x.replace(',','.'))
df_cost['Ad_group_target_CPA'] = df_cost['Ad_group_target_CPA'].astype(float)
df_cost['Ad_group_target_CPA'] = df_cost['Ad_group_target_CPA'].astype(int)
df_cost['Ad_group_target_CPA'] = df_cost['Ad_group_target_CPA'].astype(str)

df_cost['Show'] = df_cost['Show'].apply(lambda x: x.replace(',',''))
df_cost['Show'] = df_cost['Show'].astype(int)
df_cost['Cost'] = df_cost['Cost'].apply(lambda x: x.replace(',','.'))
df_cost['Cost'] = df_cost['Cost'].astype(float)
df_cost['Clicks'] = df_cost['Clicks'].astype(int)

In [20]:
# Преобразование даты
df_cost['Date'] = pd.to_datetime(df_cost['Date'])
df_cost['Date'] = df_cost['Date'].apply(lambda x: x.date())

df_result['Date'] = pd.to_datetime(df_result['Date'])
df_result['Date'] = df_result['Date'].apply(lambda x: x.date())

### Currency conversion

In [21]:
# получение курса RUB на сегодняшнюю дату
import requests
try:
    r = requests.get('https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?valcode=RUB&json').json()
    RUB_ = r[0]['rate']
    r_USD = requests.get('https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?valcode=USD&json').json()
    USD_ = r_USD[0]['rate']
except:
    RUB_ = 0.4
    USD_ = 27

In [22]:
#RUB
df_cost_rub = df_cost.loc[df_cost.Currency == 'RUB']
df_cost_rub = df_cost_rub.assign(Cost = df_cost_rub.Cost*RUB_)
df_cost_uah = df_cost.loc[df_cost.Currency != 'RUB']
f_cost = pd.concat([df_cost_uah, df_cost_rub])

In [23]:
# #USD
df_cost_usd = df_cost.loc[df_cost.Currency == 'USD']
df_cost_usd = df_cost_usd.assign(Cost = df_cost_usd.Cost*USD_)
df_cost_usd = df_cost_usd.round(decimals=2)
df_cost_uah = df_cost.loc[df_cost.Currency == 'UAH']
df_cost = pd.concat([df_cost_uah, df_cost_usd])

### Sorting

In [26]:
# Сортировка по убыванию
df_cost = df_cost.sort_values(by = ['Cost'], axis = 0, ascending=False)

In [27]:
# Сортировка от Я до А
df_result = df_result.sort_values(by = ['Action_conversion', 'All_conv'], axis = 0, ascending=[True, False])

In [28]:
df_result['Action_conversion'].unique()

array(['FTD', 'First open', 'Install', 'Registration', 'Session start'],
      dtype=object)

### Page pivot result

In [29]:
# Фильтр по конверсиям
df_result_filter = df_result[df_result['Action_conversion'].str.contains(r'\bRegistration|First open|Install|Session start|FTD\b')].copy()

In [30]:
# Удаление колонки
df_result_filter.drop(['Currency'], axis=1, inplace=True)

In [31]:
# Формирование сводной таблицы
pivot_result = df_result_filter.pivot_table(
    index=['Campaign', 'Ad_group', 'Ad_group_target_CPA', 'Network','Date'],
    columns=['Action_conversion'],
    aggfunc=np.sum,
    fill_value=0)

In [32]:
# Удаление уровня Action_conversion
pivot_result.columns = pivot_result.columns.droplevel()

### Page pivot final

In [33]:
# Объединение
df_cost_result = pd.merge(df_cost, pivot_result, 
                    how ='left',  
                    left_on=(['Campaign', 'Ad_group', 'Ad_group_target_CPA', 'Network', 'Date']), 
                    right_index=True)

In [34]:
try:
    df_cost_result = df_cost_result[['Campaign', 'Ad_group', 'Date', 'Ad_group_target_CPA', 'Network',
       'Currency', 'Cost', 'Show', 'Clicks', 'First open', 'FTD', 'Install',
       'Registration', 'Session start']]
except:
    try:
        df_cost_result['FTD'] = 0
        df_cost_result = df_cost_result[['Campaign', 'Ad_group', 'Date', 'Ad_group_target_CPA', 'Network',
           'Currency', 'Cost', 'Show', 'Clicks', 'First open', 'FTD', 'Install',
           'Registration', 'Session start']]
    except:
        df_cost_result['FTD'] = 0
        df_cost_result['Registration'] = 0
        df_cost_result = df_cost_result[['Campaign', 'Ad_group', 'Date', 'Ad_group_target_CPA', 'Network',
           'Currency', 'Cost', 'Show', 'Clicks', 'First open', 'FTD', 'Install',
           'Registration', 'Session start']]        

In [35]:
pivot_result = pivot_result.reset_index()

In [36]:
# Замена NaN
df_cost_result.replace(np.nan, 0, inplace=True)

In [37]:
df_final = df_cost_result.copy()

### Time

In [38]:
df_time = naming.copy()

In [39]:
# Переименование колонок
df_time = df_time.rename(columns=
                       {'Время прохождения модерации': 'Moderation_time',
                        'Stock_Campaign': 'Campaign',
                        'Дата запуска': 'Start_date',
                        'Время запуска': 'Start_time'
                       })

In [40]:
# Удаление пустых строк
df_time.dropna(subset=['Start_date'], inplace=True)
df_time.dropna(subset=['Start_time'], inplace=True)
df_time.dropna(subset=['Campaign'], inplace=True)

In [41]:
# Замена точки на двоеточие
df_time['Start_time'] = df_time['Start_time'].astype(str)
df_time['Start_time'] = df_time['Start_time'].apply(lambda x: x.replace('.',':'))
df_time['Start_time'] = df_time['Start_time'].apply(lambda x: x.replace(',',':'))
df_time['Start_time'] = df_time['Start_time'].apply(lambda x: x.replace(';',':'))

In [43]:
df_time['Start_date'] = pd.to_datetime(df_time['Start_date'], format='%Y-%m-%dT')

In [44]:
# Создание timestamp
df_time['DateTime'] = df_time['Start_date'].astype(str) + ' ' + df_time['Start_time'].astype(str)

In [45]:
df_time['DateTime'] = pd.to_datetime(df_time['DateTime'])
# df_time['timestamp'] = df.datetime.values.astype(np.int64) // 10 ** 9

In [49]:
# Округление времени
def rounder(t):
    if t.minute >= 30:
        return t.replace(second=0, microsecond=0, minute=0, hour=t.hour+1).strftime('%H:%M')
    else:
        return t.replace(second=0, microsecond=0, minute=0).strftime('%H:%M')

df_time['Time'] = df_time['DateTime'].map(rounder)

In [50]:
# Округление времени
def encoding_time(t):
    if t.hour >= 10 and t.hour < 13:
        return '10-13'
    elif t.hour >= 13 and t.hour < 16:
        return '13-16'
    elif t.hour >= 16 and t.hour < 19:
        return '16-19'
    elif t.hour >= 19 and t.hour <= 22:
        return '19-22'
    elif t.hour >= 0 and t.hour < 1:
        return '00-00'
    elif t.hour >= 1 and t.hour < 5:
        return '01-05'
    elif t.hour >= 5 and t.hour < 10:
        return '05-10'
    else:
        return 'unknown'
    
    
df_time['Time_code'] = df_time['DateTime'].map(encoding_time)

In [51]:
df_time['Start_date'] = df_time['Start_date'].astype(str)

In [52]:
df_time = df_time[['Campaign', 'Start_date', 'Moderation_time', 'Start_time', 'Time_code']]

In [53]:
# Без этого файл двоит
df_time.drop_duplicates(subset=['Campaign'], keep='first', inplace=True)

In [54]:
df_final = df_final.merge(df_time, how='left', on=["Campaign"])

### Split

In [55]:
df_final['Campaign_OPI'] = df_final['Campaign']
# Замена значений
df_final['Campaign_OPI'].replace(Stock_Campaign, Google_nameng, inplace=True)

In [56]:
# Обрезать до OPI
df_final['Campaign_OPI'] = df_final['Campaign_OPI'].str.findall(r'OP\S+').str[-1]
# Текст по столбцам
df_cost_split = df_final['Campaign_OPI'].str.split(r'\_|\-', expand=True)
# Переименование колонок
df_cost_split = df_cost_split.rename(columns=
                                     {0: 'Optimization',
                                      1: 'Geo',
                                      2: 'Brand',
                                      3: 'Page',
                                      4: 'ADS',
                                      5: 'Camp',
                                      6: 'Adset',
                                      7: 'Creo',
                                      8: 'txt',
                                      9: 'App',
                                      10: 'OwnerApp',
                                      11: 'Stream'
                                     })

In [57]:
df_final = pd.concat([df_final,df_cost_split],axis=1)

In [58]:
#Замена значений
df_final['Ad_group'].replace(regex='Ad group 1', value='adset1', inplace=True)
df_final['Ad_group'].replace(regex='Ad group 2', value='adset2', inplace=True)
df_final['Ad_group'].replace(regex='Ad group 3', value='adset3', inplace=True)
df_final['Ad_group'].replace(regex='Ad group 4', value='adset4', inplace=True)

In [59]:
# Удаление пробелов
df_final['Ad_group'].replace(r'\s+','',regex=True, inplace=True)

### Replace creo

In [60]:
df_final['Creo_new'] = df_final['Ad_group'].str.findall(r'-\w+').str[-1].str[1:]

In [61]:
#Замена пустых полей
# None to Nan
df_final.Creo_new.fillna(value=np.nan, inplace=True)

In [62]:
df_final['Creo_new'] = df_final['Creo_new'].fillna(df_final['Creo'])

In [63]:
# Удаление колонок
df_final.drop('Creo', axis=1, inplace=True)

In [64]:
# Переименование колонок
df_final = df_final.rename(columns= {'Creo_new': 'Creo'})

In [65]:
# Нижний регистр
df_final['Ad_group'] = df_final['Ad_group'].str.lower()

### Info App

In [66]:
# Удаление колонок
df_app.drop(['Скрин', 'Link', 'Разработчик', 'Обновлено', 'Базовое кол-во установок'], axis=1, inplace=True)

In [67]:
# Изменение форматов
df_app.replace(np.nan, 0, inplace=True)
df_app['Размер'] = df_app['Размер'].astype(int)
df_app['К-во белых прил'] = df_app['К-во белых прил'].astype(int)
df_app['К-во казиношных прил'] = df_app['К-во казиношных прил'].astype(int)

In [68]:
#Замена на Main
df_final['Stream'].replace('Main|sub3', 'Main', inplace=True)

In [69]:
df_final = df_final.merge(df_app, how='left', on=["App"])

In [70]:
# # Сортировка по возростанию
df_final = df_final.sort_values(by = ['Campaign'], axis = 0, ascending=True)

In [71]:
df_final = df_final.assign(Cost_USD =  df_final.Cost/USD_)

In [72]:
# Два знака после запятой
df_final['Cost_USD'] = df_final['Cost_USD'].round(decimals=2)

In [73]:
df_final = df_final[['Campaign', 'Ad_group', 'Date', 'Time_code', 'Ad_group_target_CPA', 'Network',
       'Currency','Cost_USD', 'Cost', 'Show', 'Clicks', 'First open', 'FTD', 'Install',
       'Registration', 'Session start', 'Optimization', 'Geo', 'Brand', 'Page',
       'ADS', 'Camp', 'Adset', 'Creo', 'txt', 'App', 'Сategory GP', 'Age limit', 'Type', 'Subtype', 'Android', 
        'Размер', 'К-во белых прил', 'К-во казиношных прил',
        'OwnerApp', 'Stream', 'Start_date', 'Moderation_time', 'Start_time']]

### Pivot table

In [76]:
df_youtube = df_final.query("Network =='Видео на YouTube'")

In [77]:
#Pivot Geo and Place
pivot = df_youtube.groupby(['Geo', 'OwnerApp', 'Optimization', 'Creo', 'Brand', 
                          'Ad_group_target_CPA', 'ADS', 'Camp', 'Ad_group', 'App', 'Date'], as_index=False).sum()

In [78]:
# Вычисление
pivot = pivot.assign(Cost_USD =  pivot.Cost/USD_)
pivot = pivot.assign(Inst_cost =  pivot.Cost_USD/ pivot.Install)
pivot = pivot.assign(Open_cost =  pivot.Cost_USD/ pivot['First open'])
pivot = pivot.assign(Reg_cost =  pivot.Cost_USD/ pivot.Registration)
pivot = pivot.assign(FTD_cost =  pivot.Cost_USD/ pivot.FTD)
pivot = pivot.assign(CPM = pivot.Cost_USD/pivot.Show*1000)
pivot = pivot.assign(CPC = pivot.Cost_USD/pivot.Clicks)
pivot = pivot.assign(CTR = pivot.Clicks/pivot.Show*100)
pivot = pivot.assign(CR_click_inst = pivot.Clicks/pivot.Install)
pivot = pivot.assign(CR_inst_open = pivot.Install/pivot['First open'])
pivot = pivot.assign(CR_open_reg = pivot['First open']/pivot.Registration)
pivot = pivot.assign(CR_reg_ftd = pivot.Registration/pivot.FTD)

# Замена Inf
pivot.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
# Два знака после запятой
pivot = pivot.round(decimals=2)

In [79]:
pivot['Ad_group_target_CPA'] = pivot['Ad_group_target_CPA'].astype(int)

In [80]:
pivot = pivot.query("Cost > 0")

In [81]:
# CPM сегмент
def cpa_segment(el):
    if el <= 40 and el != 0:
        return '0 - 40'
    elif el == 0:
        return 'Auto'
    elif el >= 41 and el <= 50:
        return '41 - 50'
    elif el >= 51 and el <= 60:
        return '51 - 60'
    elif el >= 61 and el <= 70:
        return '61 - 70'
    elif el >= 71 and el <= 80:
        return '71 - 80'
    elif el >= 81 and el <= 90:
        return '81 - 90'
    elif el >= 91 and el <= 100:
        return '91 - 100'
    elif el >= 101 and el <= 110:
        return '101 - 110'
    elif el >= 111 and el <= 120:
        return '111 - 120'
    elif el >= 121 and el <= 130:
        return '121 - 130'
    elif el >= 131 and el <= 140:
        return '131 - 140'
    elif el >= 141 and el <= 150:
        return '141 - 150'
    elif el >= 150 and el <= 999:
        return el
    else:
        return 'Больше 1000'

In [82]:
# CPM сегмент
def cpm_segment(el):
    if el <= 3 and el != 0:
        return '0 - 3.00'
    elif el == 0:
        return 'Нет конверсий'
    elif el >= 3.01 and el <= 4:
        return '3.01 - 4.00'
    elif el >= 4.01 and el <= 5:
        return '4.01 - 5.00'
    elif el >= 5.01 and el <= 6:
        return '5.01 - 6.00'
    else:
        return 'Больше 6.00'

In [83]:
# CPC сегмент
def cpc_segment(el):
    if el <= 0.10 and el!=0:
        return '0 - 0.10'
    elif el == 0:
        return 'Нет конверсий'
    elif el >= 0.11 and el <= 0.20:
        return '0,11 - 0,20'
    elif el >= 0.21 and el <= 0.30:
        return '0.21 - 0.30'
    elif el >= 0.31 and el <= 0.40:
        return '0.31 - 0.40'
    elif el >= 0.41 and el <= 0.50:
        return '0.41 - 0.50'
    elif el >= 0.51 and el <= 0.60:
        return '0.51 - 0.60'
    elif el >= 0.61 and el <= 0.70:
        return '0.61 - 0.70'
    elif el >= 0.71 and el <= 0.80:
        return '0,71 - 0,80'
    elif el >= 0.81 and el <= 0.90:
        return '0.81 - 0.90'
    elif el >= 0.91 and el <= 1.00:
        return '0.91 - 1.00'
    else:
        return 'Больше 1.01'

In [84]:
# Inst.cost сегмент
def inst_cost_segment(el):
    if el <= 1 and el!=0:
        return '0.00 - 1.00'
    elif el == 0:
        return 'Нет конверсий'
    elif el >= 1 and el <= 1.5:
        return '1.00 - 1.50'
    elif el >= 1.51 and el <= 2:
        return '1.51 - 2.00'
    elif el >= 2.01 and el <= 2.50:
        return '2.01 - 2.50'
    elif el >= 2.51 and el <= 3.00:
        return '2.51 - 3.00'
    else:
        return 'Больше 3.01'

In [85]:
# Inst.cost сегмент
def inst_cost_segment_AU_DE(el):
    if el <= 3 and el!=0:
        return '0.00 - 3.00'
    elif el == 0:
        return 'Нет конверсий'
    elif el >= 3.01 and el <= 6:
        return '3.01 - 6.00'
    elif el >= 6.01 and el <= 9:
        return '6.01 - 9.00'
    elif el >= 9.01 and el <= 12:
        return '9.01 - 12.00'
    elif el >= 12.01 and el <= 15:
        return '12.01 - 15.00'
    elif el >= 15.01 and el <= 19:
        return '15.01 - 19.00'
    else:
        return 'Больше 19.01'

In [86]:
# Reg cost сегмент
def reg_cost_segment(el):
    if el <= 2 and el!=0:
        return '0.00 - 2.00'
    elif el == 0:
        return 'Нет конверсий'
    elif el >= 2.01 and el <= 3:
        return '2.01 - 3.00'
    elif el >= 3.01 and el <= 4:
        return '3.01 - 4.00'
    elif el >= 4.01 and el <= 5:
        return '4.01 - 5.00'
    elif el >= 5.01 and el <= 6:
        return '5.01 - 6.01'
    elif el >= 7.01 and el <= 8:
        return '7.01 - 8.00'
    elif el >= 8.01 and el <= 9:
        return '8.01 - 9.00'
    elif el >= 9.01 and el <= 10:
        return '9.01 - 10.00'
    else:
        return 'Больше 10.01'

In [87]:
# Reg cost сегмент
def reg_cost_segment_AU_DE(el):
    if el <= 5 and el!=0:
        return '0.00 - 5.00'
    elif el == 0:
        return 'Нет конверсий'
    elif el >= 5.01 and el <= 10:
        return '5.01 - 10.00'
    elif el >= 10.01 and el <= 15:
        return '10.01 - 15.00'
    elif el >= 15.01 and el <= 20:
        return '15.01 - 20.00'
    elif el >= 20.01 and el <= 25:
        return '20.01 - 25.00'
    elif el >= 25.01 and el <= 30:
        return '25.01 - 30.00'
    elif el >= 30.01 and el <= 35:
        return '30.01 - 35.00'
    elif el >= 35.01 and el <= 40:
        return '35.01 - 40.00'
    else:
        return 'Больше 40.01'

In [88]:
#  CTR сегмент
def CTR_segment(el):
    if el <= 0.10 and el!=0:
        return '0.00% - 0.10%'
    elif el == 0:
        return 'Нет конверсий'
    elif el >= 0.11 and el <= 0.20:
        return '0.11% - 0.20%'
    elif el >= 0.21 and el <= 0.30:
        return '0.21% - 0.30%'
    elif el >= 0.31 and el <= 0.40:
        return '0.31% - 0.40%'
    elif el >= 0.41 and el <= 0.50:
        return '0.41% - 0.50%'
    elif el >= 0.51 and el <= 0.60:
        return '0.51% - 0.60%'
    elif el >= 0.61 and el <= 0.70:
        return '0.61% - 0.70%'
    elif el >= 0.71 and el <= 0.80:
        return '0.71% - 0.80%'
    elif el >= 0.81 and el <= 0.90:
        return '0.81% - 0.90%'
    elif el >= 0.91 and el <= 1.00:
        return '0.91% - 1.00%'
    elif el >= 1.01 and el <= 1.10:
        return '1.01 - 1.10'
    elif el >= 1.11 and el <= 1.20:
        return '1.11 - 1.20'
    elif el >= 1.21 and el <= 1.30:
        return '1.21 - 1.30'
    elif el >= 1.31 and el <= 1.40:
        return '1.31 - 1.40'
    elif el >= 1.41 and el <= 1.50:
        return '1.41 - 1.50'
    else:
        return 'Больше 1.51%'

In [89]:
# Open.cost сегмент
def open_cost_segment(el):
    if el <= 1 and el!=0:
        return '0.00 - 1.00'
    elif el == 0:
        return 'Нет конверсий'
    elif el >= 1 and el <= 1.5:
        return '1.00 - 1.50'
    elif el >= 1.51 and el <= 2:
        return '1.51 - 2.00'
    elif el >= 2.01 and el <= 2.50:
        return '2.01 - 2.50'
    elif el >= 2.51 and el <= 3.00:
        return '2.51 - 3.00'
    else:
        return 'Больше 3.01'

In [90]:
# Inst.cost сегмент
def open_cost_segment_AU_DE(el):
    if el <= 3 and el!=0:
        return '0.00 - 3.00'
    elif el == 0:
        return 'Нет конверсий'
    elif el >= 3.01 and el <= 6:
        return '3.01 - 6.00'
    elif el >= 6.01 and el <= 9:
        return '6.01 - 9.00'
    elif el >= 9.01 and el <= 12:
        return '9.01 - 12.00'
    elif el >= 12.01 and el <= 15:
        return '12.01 - 15.00'
    elif el >= 15.01 and el <= 19:
        return '15.01 - 19.00'
    elif el >= 19.01 and el <= 22:
        return '19.01 - 22.00'
    elif el >= 22.01 and el <= 25:
        return '22.01 - 25.00'
    else:
        return 'Больше 19.01'

In [91]:
# CR click/inst сегмент
def CR_click_inst_segment(el):
    if el <= 1.5 and el!=0:
        return '0.00 - 1.50'
    elif el == 0:
        return 'Нет конверсий'
    elif el >= 1.51 and el <= 2:
        return '1.51 - 2.00'
    elif el >= 2.01 and el <= 2.5:
        return '2.01 - 2.50'
    elif el >= 2.51 and el <= 3:
        return '2.51 - 3.00'
    elif el >= 3.01 and el <= 3.5:
        return '3.01 - 3.50'
    elif el >= 3.51 and el <= 4:
        return '3.51 - 4.00'
    elif el >= 4.01 and el <= 5:
        return '4.01 - 5.00'
    else:
        return 'Больше 5.01'

In [92]:
#  CR open/reg сегмент
def CR_open_reg_segment(el):
    if el <= 1 and el!=0:
        return '0.00 - 1.00'
    elif el == 0:
        return 'Нет конверсий'
    elif el >= 1.01 and el <= 1.5:
        return '1.01 - 1.50'
    elif el >= 1.51 and el <= 2:
        return '1.51 - 2.00'
    elif el >= 2.01 and el <= 2.5:
        return '2.01 - 2.50'
    else:
        return 'Больше 2.51'

In [93]:
#  CR reg/ftd сегмент
def CR_reg_ftd_segment(el):
    if el <= 5 and el!=0:
        return '0.00 - 5.00'
    elif el == 0:
        return 'Нет конверсий'
    elif el >= 5.01 and el < 10:
        return '5.01 - 10.00'
    else:
        return 'Больше 10.01'

In [94]:
#  CTR сегмент
def CTR_cost_segment(el):
    if el <= 1 and el!=0:
        return '0.00% - 1.00%'
    elif el >= 1.01 and el <= 1.5:
        return '1.01% - 1.50%'
    elif el >= 1.51 and el <= 2:
        return '1.51% - 2.00%'
    elif el >= 2 and el <= 2.5:
        return '2.01% - 2.50%'
    elif el >= 2.5 and el <= 3:
        return '2.51% - 3.00%'
    else:
        return '3.00% и больше'

In [95]:
# CR inst/open сегмент  
def CR_inst_open_segment(el):
    if el <= 1 and el!=0:
        return '0.00 - 1.00'
    elif el >= 1.01 and el <= 1.5:
        return '1.01 - 1.50'
    elif el >= 1.51 and el <= 2:
        return '1.51 - 2.00'
    elif el >= 2.01 and el <= 2.5:
        return '2.01 - 2.50'
    elif el >= 2.51 and el <= 3:
        return '2.51 - 3.00'
    else:
        return '3.01 и больше'

In [96]:
pivot['CPA_segment'] = pivot['Ad_group_target_CPA'].map(cpa_segment)
pivot['CPM_segment'] = pivot['CPM'].map(cpm_segment)
pivot['CPC_segment'] = pivot['CPC'].map(cpc_segment)
pivot['Inst_cost_segment'] = pivot['Inst_cost'].map(inst_cost_segment)
pivot['Inst_cost_segment_AU_DE'] = pivot['Inst_cost'].map(inst_cost_segment_AU_DE)
pivot['Open_cost_segment'] = pivot['Open_cost'].map(open_cost_segment)
pivot['Open_cost_segment_AU_DE'] = pivot['Open_cost'].map(open_cost_segment_AU_DE)
pivot['Reg_cost_segment'] = pivot['Reg_cost'].map(reg_cost_segment)
pivot['Reg_cost_segment_AU_DE'] = pivot['Reg_cost'].map(reg_cost_segment_AU_DE)
pivot['CTR_cost_segment'] = pivot['CTR'].map(CTR_segment)
pivot['CR_click_inst_segment'] = pivot['CR_click_inst'].map(CR_click_inst_segment)
pivot['CR_inst_open_segment'] = pivot['CR_inst_open'].map(CR_inst_open_segment)
pivot['CR_open_reg_segment'] = pivot['CR_open_reg'].map(CR_open_reg_segment)
pivot['CR_reg_ftd_segment'] = pivot['CR_reg_ftd'].map(CR_reg_ftd_segment)

In [97]:
pivot = pivot[['Geo', 'OwnerApp', 'Optimization', 'Creo', 'Brand', 'Ad_group_target_CPA', 'ADS', 'Camp', 'Ad_group',
      'App', 'Date','Cost_USD', 'Cost', 'Show', 'Clicks', 'Install', 'First open', 
      'Registration', 'FTD', 'CPM', 'CPM_segment', 'CPC', 'CPC_segment', 
      'Inst_cost', 'Inst_cost_segment', 'Inst_cost_segment_AU_DE', 'Open_cost', 'Open_cost_segment', 'Open_cost_segment_AU_DE',
      'Reg_cost', 'Reg_cost_segment', 'Reg_cost_segment_AU_DE', 'FTD_cost', 'CTR', 'CTR_cost_segment', 
      'CR_click_inst', 'CR_click_inst_segment', 'CR_inst_open', 'CR_inst_open_segment',
      'CR_open_reg', 'CR_open_reg_segment', 'CR_reg_ftd', 'CR_reg_ftd_segment', 'CPA_segment']]

In [98]:
# Только номер креатива
pivot['Creo_group'] = pivot['Creo'].str.split('(\d+)([A-Za-z]+)', expand=True).loc[:,[1]]
df_final['Creo_group'] = df_final['Creo'].str.split('(\d+)([A-Za-z]+)', expand=True).loc[:,[1]]

## Standardization

In [99]:
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
import pickle

In [100]:
attrs_num = ['CPM', 'CPC', 'CTR', 'Inst_cost', 'Open_cost', 'Reg_cost', 'CR_click_inst', 'CR_inst_open', 'CR_open_reg']
scl = StandardScaler()
x_num = pd.DataFrame(scl.fit_transform(pivot[attrs_num]), columns=attrs_num)

In [101]:
train_data = x_num.copy()

In [102]:
model = xgb.XGBClassifier()
model.load_model("model.json")

In [103]:
pred = model.predict(train_data)

In [104]:
pred_series = pd.Series(pred, name='Prediction')

In [105]:
pivot = pd.concat([pivot, pred_series], axis=1)

In [106]:
with open("GaussianNB.pkl", 'rb') as open_file:
    loaded_classifier = pickle.load(open_file)

print(loaded_classifier)

GaussianNB(priors=[0.9, 0.1])


In [107]:
pred_g = loaded_classifier.predict(train_data)

In [108]:
pred_series_G = pd.Series(pred_g, name='Optimistic_prediction')

In [109]:
pivot = pd.concat([pivot, pred_series_G], axis=1)

### Save File

In [110]:
writer = pd.ExcelWriter('Analysis_Сonversion.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_final.to_excel(writer, sheet_name='Pivot', index=False)
pivot.to_excel(writer, sheet_name='Segment', index=False)


# # Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet_Pivot = writer.sheets['Pivot']

# Number Format Categorie
currency_format = workbook.add_format({'num_format': 
                                       '_-* # ##0_-;-* # ##0_-;_-* "-"??_-;_-@_-'})
center = workbook.add_format({'align': 'center'})

# # Set the column width and format.
worksheet_Pivot.set_column('A:C', 12)
worksheet_Pivot.set_column('W:W', 20)
worksheet_Pivot.set_column('Y:AH', 12)
worksheet_Pivot.set_column('G:G', None, center)
writer.save()

In [111]:
print('Готово')

Готово
